<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Hands-on Lab: Interactive Visual Analytics with Folium**


Estimated time needed: **40** minutes


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


This lab contains the following tasks:

*   **TASK 1:** Mark all launch sites on a map
*   **TASK 2:** Mark the success/failed launches for each site on the map
*   **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [1]:
import piplite
await piplite.install(['folium'])
await piplite.install(['pandas'])

In [2]:
import folium
import pandas as pd

In [3]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/labs/v4/DV0101EN-Exercise-Generating-Maps-in-Python.ipynb)


In [4]:
# Create a base map centered at a global view (latitude and longitude are chosen arbitrarily)
map_launch_sites = folium.Map(location=[20, 0], zoom_start=2)  # Base map centered roughly at the equator

# Define a dictionary with launch site names and their approximate coordinates (latitude, longitude)
launch_sites_coords = {
    "CCAFS SLC 40": [28.562106, -80.57718],
    "VAFB SLC 4E": [34.632093, -120.610829],
    # Ha több kilövési hely is szerepel a datasetben, add hozzá őket itt
}

# Iterate over the dictionary and add a marker for each launch site on the map
for site, coords in launch_sites_coords.items():
    folium.Marker(
        location=coords,       # Marker coordinates
        popup=site,            # Popup text displays the launch site name
        icon=folium.Icon(color='blue')  # Optionally, set a marker color
    ).add_to(map_launch_sites)

# Display the interactive map with all launch sites marked
map_launch_sites




First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


In [5]:
# Download and read the spacex_launch_geo.csv file
from js import fetch        # Import fetch function for downloading data from a URL
import io                   # Import io module for handling byte streams

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
resp = await fetch(URL)      # Asynchronously fetch the CSV file from the given URL
spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())  # Convert the response into a Python byte stream
spacex_df = pd.read_csv(spacex_csv_file)  # Read the CSV data into a pandas DataFrame

spacex_df.head()  # Display the first few rows of the DataFrame


,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Failure (parachute),0,28.562302,-80.577356
1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",0.0,LEO (ISS),NASA (COTS) NRO,Failure (parachute),0,28.562302,-80.577356
2,3,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2+,525.0,LEO (ISS),NASA (COTS),No attempt,0,28.562302,-80.577356
3,4,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356
4,5,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356


Now, you can take a look at what are the coordinates for each site.


In [6]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [7]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,


In [8]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle.


Now, let's add a circle for each launch site in data frame `launch_sites`


*TODO:*  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


An example of folium.Marker:


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


In [9]:
# Loop through each launch site in the launch_sites_df DataFrame and add a Circle and a Marker to the map
for index, row in launch_sites_df.iterrows():
    # Create a coordinate list from latitude and longitude values
    coordinate = [row['Lat'], row['Long']]
    
    # Create a Circle for the launch site with a popup displaying the launch site name
    circle = folium.Circle(
        location=coordinate,         # Center of the circle: the launch site's coordinate
        radius=1000,                 # Radius of the circle in meters
        color='#d35400',             # Border color of the circle (orange)
        fill=True,                   # Fill the circle with color
        fill_color='#d35400'         # Set the fill color to orange as well
    ).add_child(folium.Popup(row['Launch Site']))  # Attach a popup with the launch site name
    
    # Create a Marker with a custom DivIcon to display the launch site name as a label
    marker = folium.map.Marker(
        location=coordinate,         # Marker location: the launch site's coordinate
        icon=DivIcon(
            icon_size=(20, 20),      # Size of the icon
            icon_anchor=(0, 0),      # Anchor point of the icon
            html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % row['Launch Site']
            # HTML code to render the launch site name in the icon in orange
        )
    )
    
    # Add both the Circle and the Marker to the map
    site_map.add_child(circle)
    site_map.add_child(marker)

# Display the updated map with the markers and circles
site_map


The generated map with marked launch sites should look similar to the following:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png">
</center>


Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:

*   Are all launch sites in proximity to the Equator line?
*   Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.


Are all launch sites in proximity to the Equator line?
No, not all launch sites are located near the Equator. While some launch sites, such as those at Cape Canaveral or Kennedy Space Center, are situated in relatively low latitudes (around 28°–29° North), they are still far from the Equator (0° latitude). Therefore, although they are in the subtropics, they are not considered to be in close proximity to the Equator.

Are all launch sites in very close proximity to the coast?
Most launch sites are indeed located near coastlines. Being close to the coast is beneficial for safety and optimal flight trajectories, as it allows for over-water launch paths that minimize risk to populated areas. However, not all launch sites are directly on the coast. Some sites may be slightly inland, but generally, they are positioned to ensure both operational efficiency and safety.

Explanation of Findings:

Latitude: The launch sites are chosen based on a mix of factors such as orbital mechanics and weather conditions. Launching from a location that is not exactly on the Equator means that the boost in rotational speed provided by the Earth is less optimal compared to an equatorial launch. However, many U.S. launch sites are in Florida or California because they provide a good balance between safety, infrastructure, and environmental conditions.
Proximity to the Coast: Launch sites are generally placed near the coast to facilitate flight paths over open water. This minimizes risk in case of an anomaly during launch and supports the correct orbital insertion. While some sites are directly adjacent to the coastline, others are a short distance inland due to geographical or operational constraints.

In [10]:
# Loop through each launch event in the spacex_df DataFrame and add a marker with a color indicating success or failure
for index, row in spacex_df.iterrows():
    # Create a coordinate list from latitude and longitude values
    coordinate = [row['Lat'], row['Long']]
    
    # Determine the marker color and popup text based on the launch outcome ('class' column)
    if row['class'] == 1:
        marker_color = 'green'  # Green for successful launches
        popup_text = f"{row['Launch Site']} - Success"
    else:
        marker_color = 'red'    # Red for failed launches
        popup_text = f"{row['Launch Site']} - Failure"
    
    # Create a CircleMarker at the launch event coordinate with the appropriate color and popup label
    folium.CircleMarker(
        location=coordinate,     # The coordinates of the launch event
        radius=6,                # Radius of the marker in pixels
        color=marker_color,      # Border color of the marker
        fill=True,               # Fill the marker with color
        fill_color=marker_color, # Fill color of the marker
        fill_opacity=0.7,        # Opacity of the fill
        popup=popup_text         # Popup text showing the launch site and outcome
    ).add_to(site_map)

# Display the interactive map (in Jupyter Notebook, simply evaluating site_map will render the map)
site_map



Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [11]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [12]:
marker_cluster = MarkerCluster()


*TODO:* Create a new column in `spacex_df` dataframe called `marker_color` to store the marker colors based on the `class` value


In [13]:
# Create a new column 'marker_color' in the spacex_df DataFrame based on the value in the 'class' column.
# If class == 1, then marker_color will be 'green'; if class == 0, then marker_color will be 'red'.
spacex_df['marker_color'] = spacex_df['class'].apply(lambda x: 'green' if x == 1 else 'red')

# Display the updated DataFrame to verify the new column
spacex_df.head()


,Launch Site,Lat,Long,class,marker_color
0,CCAFS LC-40,28.562302,-80.577356,0,red
1,CCAFS LC-40,28.562302,-80.577356,0,red
2,CCAFS LC-40,28.562302,-80.577356,0,red
3,CCAFS LC-40,28.562302,-80.577356,0,red
4,CCAFS LC-40,28.562302,-80.577356,0,red


*TODO:* For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [14]:
#######################################
# 1. Create the base map
#######################################

# Choose a center coordinate and zoom level (e.g., NASA JSC or a specific launch site)
nasa_coordinate = [29.559684888503615, -95.0830971930759]  # NASA Johnson Space Center
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

#######################################
# 2. Draw orange circles and markers for each launch site
#######################################

# Assume launch_sites_df has the following columns:
# "Launch Site", "Lat", "Long"

for index, row in launch_sites_df.iterrows():
    coordinate = [row['Lat'], row['Long']]
    
    # Create an orange circle around the launch site
    circle = folium.Circle(
        location=coordinate,
        radius=1000,              # Circle radius in meters
        color='#d35400',          # Circle border color (orange)
        fill=True,
        fill_color='#d35400',     # Circle fill color (orange)
        fill_opacity=0.3
    ).add_child(folium.Popup(row['Launch Site']))
    
    # Create an orange label for the launch site name
    marker = folium.map.Marker(
        location=coordinate,
        icon=DivIcon(
            icon_size=(20, 20),
            icon_anchor=(0, 0),
            html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % row['Launch Site']
        )
    )
    
    site_map.add_child(circle)
    site_map.add_child(marker)

#######################################
# 3. Create a MarkerCluster and add launch events
#######################################

# Assume spacex_df has columns:
# "Lat", "Long", "Launch Site", "class", and "marker_color" (e.g., 'green' for success, 'red' for failure)

marker_cluster = MarkerCluster().add_to(site_map)

for index, row in spacex_df.iterrows():
    coordinate = [row['Lat'], row['Long']]
    
    # Determine the launch outcome
    launch_outcome = 'Success' if row['class'] == 1 else 'Failure'
    popup_text = f"{row['Launch Site']} - {launch_outcome}"
    
    marker = folium.Marker(
        location=coordinate,
        popup=popup_text,
        icon=folium.Icon(color=row['marker_color'])  # 'green' for success, 'red' for failure
    )
    
    marker_cluster.add_child(marker)

#######################################
# 4. Display the final map
#######################################

site_map


Your updated map may look like the following screenshots:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png">
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png">
</center>


From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


In [15]:
# Merge spacex_df with launch_sites_df on "Launch Site" to obtain official site coordinates for each event
merged_df = spacex_df.merge(launch_sites_df, on="Launch Site", suffixes=('_event', '_site'))

# Import necessary math functions for the Haversine formula
from math import radians, sin, cos, sqrt, atan2

# Define the haversine function to calculate the distance between two geographic points
def haversine(lat1, lon1, lat2, lon2):
    R = 6371.0  # Earth's radius in kilometers
    # Convert degrees to radians
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = R * c
    return distance

# Apply the haversine function to each row of the merged DataFrame to calculate the distance
merged_df['distance_km'] = merged_df.apply(
    lambda row: haversine(row['Lat_event'], row['Long_event'], row['Lat_site'], row['Long_site']), axis=1
)

# Display the first few rows of the merged DataFrame, which now includes the calculated distances
merged_df.head()


,Launch Site,Lat_event,Long_event,class,marker_color,Lat_site,Long_site,distance_km
0,CCAFS LC-40,28.562302,-80.577356,0,red,28.562302,-80.577356,0.0
1,CCAFS LC-40,28.562302,-80.577356,0,red,28.562302,-80.577356,0.0
2,CCAFS LC-40,28.562302,-80.577356,0,red,28.562302,-80.577356,0.0
3,CCAFS LC-40,28.562302,-80.577356,0,red,28.562302,-80.577356,0.0
4,CCAFS LC-40,28.562302,-80.577356,0,red,28.562302,-80.577356,0.0


Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [16]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


In [17]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

*TODO:* Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [18]:
# Example launch site coordinates (e.g., for CCAFS SLC-40)
launch_site_lat = 28.563197  # Latitude of the launch site
launch_site_lon = -80.576820 # Longitude of the launch site

# Coastline coordinate (as provided)
coastline_lat = 28.56332     # Latitude of the closest coastline point
coastline_lon = -80.56798    # Longitude of the closest coastline point

# Calculate the distance between the launch site and the coastline point
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)
print("Distance from launch site to coastline: {:.2f} km".format(distance_coastline))

# Mark the coastline point on the map with a marker
coastline_marker = folium.Marker(
    location=[coastline_lat, coastline_lon],  # Coordinates of the coastline point
    popup=f"Coastline\nDistance: {distance_coastline:.2f} km",  # Popup showing the distance
    icon=folium.Icon(color='blue')  # Blue icon for the coastline marker
)
site_map.add_child(coastline_marker)

# Display the updated map with the coastline marker
site_map


Distance from launch site to coastline: 0.86 km


In [36]:
# Create a folium.Marker to display the distance between the launch site and the closest coastline point.
distance_marker = folium.Marker(
    location=[coastline_lat, coastline_lon],  # Coordinates of the coastline point
    icon=DivIcon(
        icon_size=(20, 20),   # Size of the icon
        icon_anchor=(0, 0),   # Anchor point for the icon
        html='<div style="font-size: 12; color:#d35400;"><b>{:10.2f} KM</b></div>'.format(distance_coastline)
        # Display the distance in km, formatted to two decimal places, in orange color
    )
)

# Add the distance marker to the map
site_map.add_child(distance_marker)

# Display the updated map
site_map


*TODO:* Draw a `PolyLine` between a launch site to the selected coastline point


Your updated map with distance line should look like the following screenshot:


In [19]:
# Create a coordinate list with the launch site and coastline point coordinates
coordinates = [[launch_site_lat, launch_site_lon], [coastline_lat, coastline_lon]]

# Create a PolyLine object using the coordinates, with a weight of 1 and a blue color
lines = folium.PolyLine(locations=coordinates, weight=1, color='blue')

# Add the PolyLine to the map
site_map.add_child(lines)

# Display the updated map
site_map


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png">
</center>


*TODO:* Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>


In [ ]:
# Create a marker with distance to a closest city, railway, highway, etc.
# Draw a line between the marker to the launch site


In [21]:
# Define the launch site coordinates (example: CCAFS SLC-40)
launch_site_lat = 28.563197   # Launch site latitude
launch_site_lon = -80.576820  # Launch site longitude

# Define coordinates for the nearest city, railway, and highway (provided values)
closest_city_lat = 28.570000    # Nearest city latitude
closest_city_lon = -80.580000   # Nearest city longitude

closest_railway_lat = 28.565500   # Nearest railway latitude
closest_railway_lon = -80.575500  # Nearest railway longitude

closest_highway_lat = 28.562500   # Nearest highway latitude
closest_highway_lon = -80.574000  # Nearest highway longitude

# Calculate distances using the calculate_distance function
distance_city = calculate_distance(launch_site_lat, launch_site_lon, closest_city_lat, closest_city_lon)
distance_railway = calculate_distance(launch_site_lat, launch_site_lon, closest_railway_lat, closest_railway_lon)
distance_highway = calculate_distance(launch_site_lat, launch_site_lon, closest_highway_lat, closest_highway_lon)

print("Distance to closest city: {:.2f} km".format(distance_city))
print("Distance to closest railway: {:.2f} km".format(distance_railway))
print("Distance to closest highway: {:.2f} km".format(distance_highway))

###############################################
# Create a marker and draw a line to the closest city
###############################################
city_marker = folium.Marker(
    location=[closest_city_lat, closest_city_lon],
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>{:10.2f} KM</b></div>'.format(distance_city)
    )
)
site_map.add_child(city_marker)

city_line = folium.PolyLine(
    locations=[[launch_site_lat, launch_site_lon], [closest_city_lat, closest_city_lon]],
    weight=1,
    color='blue'
)
site_map.add_child(city_line)

###############################################
# Create a marker and draw a line to the closest railway
###############################################
railway_marker = folium.Marker(
    location=[closest_railway_lat, closest_railway_lon],
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>{:10.2f} KM</b></div>'.format(distance_railway)
    )
)
site_map.add_child(railway_marker)

railway_line = folium.PolyLine(
    locations=[[launch_site_lat, launch_site_lon], [closest_railway_lat, closest_railway_lon]],
    weight=1,
    color='blue'
)
site_map.add_child(railway_line)

###############################################
# Create a marker and draw a line to the closest highway
###############################################
highway_marker = folium.Marker(
    location=[closest_highway_lat, closest_highway_lon],
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>{:10.2f} KM</b></div>'.format(distance_highway)
    )
)
site_map.add_child(highway_marker)

highway_line = folium.PolyLine(
    locations=[[launch_site_lat, launch_site_lon], [closest_highway_lat, closest_highway_lon]],
    weight=1,
    color='blue'
)
site_map.add_child(highway_line)

# Display the final map
site_map


Distance to closest city: 0.82 km
Distance to closest railway: 0.29 km
Distance to closest highway: 0.29 km


Based on the distance lines and markers plotted on the map, you can generally conclude the following:

Railways:
Launch sites do not appear to be extremely close to railways. They might be within a reasonable distance for logistical purposes, but not immediately adjacent.

Highways:
Similarly, launch sites are not right next to highways; they are moderately proximate but still maintain some distance.

Coastline:
Launch sites are usually located very close to coastlines. This proximity is beneficial for safety reasons, as launches can be directed over open water.

Cities:
Launch sites tend to be situated at a certain distance away from cities. This separation is likely due to safety regulations and the need to minimize risks in case of launch anomalies.

After you plot distance lines to the proximities, you can answer the following questions easily:

*   Are launch sites in close proximity to railways?
*   Are launch sites in close proximity to highways?
*   Are launch sites in close proximity to coastline?
*   Do launch sites keep certain distance away from cities?

Also please try to explain your findings.


# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.


## Authors


[Pratiksha Verma](https://www.linkedin.com/in/pratiksha-verma-6487561b1/)


<!--## Change Log--!>


<!--| Date (YYYY-MM-DD) | Version | Changed By      | Change Description      |
| ----------------- | ------- | -------------   | ----------------------- |
| 2022-11-09        | 1.0     | Pratiksha Verma | Converted initial version to Jupyterlite|--!>


### <h3 align="center"> IBM Corporation 2022. All rights reserved. <h3/>
